In [1]:
import sklearn_crfsuite

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from helper.dataset_reader import read_tsv
from helper.features import feature_extraction_basic

# import warnings filter
from warnings import simplefilter

# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [2]:
!pip install -U 'scikit-learn<0.24'

You should consider upgrading via the '/Users/ahmadfathanhidayatullah/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
data = read_tsv('../raw dataset/all-tagged-090422-merged.tsv')

In [4]:
X = []
y = []

for tokens, tags in data[0]:
    X.append(tokens)
    y.append(tags)

In [5]:
X_train_, X_test_, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

In [6]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',  # for gradient descent for optimization and getting model parameters
    c1=0.1, # Coefficient for Lasso (L1) regularization
    c2=0.1, # Coefficient for Ridge (L2) regularization
    max_iterations=100, # The maximum number of iterations for optimization algorithms, iteration for the gradient descent optimization
    all_possible_transitions=True   # Specify whether CRFsuite generates transition features that do not even occur in the training data
)

In [7]:
X_train = []
for tokens in X_train_:
    X_train.append(feature_extraction_basic(tokens, 5))

#try:
crf.fit(X_train, y_train)
#except AttributeError:
#    pass

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [8]:
labels = ['ID', 'JV', 'EN', 'NE', 'O', 'MIX-ID-EN', 'MIX-ID-JV', 'MIX-JV-EN']

In [9]:
X_test = []
for tokens in X_test_:
    X_test.append(feature_extraction_basic(tokens, 5))

y_pred = crf.predict(X_test)
print('Testing Data Performance')
flat_y = [item for y_ in y_test for item in y_]
flat_y_pred = [item for y_pred_ in y_pred for item in y_pred_]
print(classification_report(flat_y, flat_y_pred, labels=labels))

Testing Data Performance
              precision    recall  f1-score   support

          ID       0.88      0.97      0.92     14891
          JV       0.85      0.85      0.85      4294
          EN       0.93      0.68      0.79      2076
          NE       0.90      0.38      0.54       900
           O       0.97      0.94      0.96      5909
   MIX-ID-EN       0.99      0.85      0.91       771
   MIX-ID-JV       0.98      0.82      0.89       256
   MIX-JV-EN       0.95      0.82      0.88       227

    accuracy                           0.90     29324
   macro avg       0.93      0.79      0.84     29324
weighted avg       0.90      0.90      0.90     29324



In [15]:
import eli5
eli5.show_weights(crf)
